In [30]:
import pandas as pd
import numpy as np

In [7]:
data = pd.read_csv('/content/drug200.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Age          200 non-null    int64  
 1   Sex          200 non-null    object 
 2   BP           200 non-null    object 
 3   Cholesterol  200 non-null    object 
 4   Na_to_K      200 non-null    float64
 5   Drug         200 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 9.5+ KB


In [16]:
data.head()

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,DrugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,DrugY


In [17]:
# Chuyển đổi các cột text thành số
data['Sex'] = data['Sex'].map({'F': 0, 'M': 1})
data['BP'] = data['BP'].map({'LOW': 0, 'NORMAL': 1, 'HIGH': 2})
data['Cholesterol'] = data['Cholesterol'].map({'NORMAL': 0, 'HIGH': 1})

# Hiển thị lại dữ liệu sau khi chuyển đổi
print(data.head())

   Age  Sex  BP  Cholesterol  Na_to_K   Drug
0   23    0   2            1   25.355  DrugY
1   47    1   0            1   13.093  drugC
2   47    1   0            1   10.114  drugC
3   28    0   1            1    7.798  drugX
4   61    0   0            1   18.043  DrugY


In [18]:
# Tách dữ liệu thành X (input features) và y (labels)
X = data.drop('Drug', axis=1)
y = data['Drug']

In [28]:
# Hàm chia node thành 2 node con
def split_node(column, threshold_split):
    left_node = column[column <= threshold_split].index
    right_node = column[column > threshold_split].index
    return left_node, right_node

# Hàm tính entropy
def entropy(y_target):
    values, counts = np.unique(y_target, return_counts=True)
    result = -np.sum([(count / len(y_target)) * np.log2(count / len(y_target)) for count in counts])
    return result

# Hàm tính information gain
def info_gain(column, target, threshold_split):
    entropy_start = entropy(target)
    left_node, right_node = split_node(column, threshold_split)
    n_target = len(target)
    n_left = len(left_node)
    n_right = len(right_node)
    entropy_left = entropy(target[left_node])
    entropy_right = entropy(target[right_node])
    weight_entropy = (n_left / n_target) * entropy_left + (n_right / n_target) * entropy_right
    ig = entropy_start - weight_entropy
    return ig

# Hàm tìm feature và threshold tốt nhất để chia
def best_split(dataX, target, feature_id):
    best_ig = -1
    best_feature = None
    best_threshold = None
    for _id in feature_id:
        column = dataX.iloc[:, _id]
        thresholds = set(column)
        for threshold in thresholds:
            ig = info_gain(column, target, threshold)
            if ig > best_ig:
                best_ig = ig
                best_feature = dataX.columns[_id]
                best_threshold = threshold
    return best_feature, best_threshold

# Hàm lấy giá trị xuất hiện nhiều nhất trong node lá
def most_value(y_target):
    return y_target.value_counts().idxmax()

# Lớp Node đại diện cho từng node trong cây
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None

# Lớp Decision Tree Classification
class DecisionTreeClass:
    def __init__(self, min_samples_split=2, max_depth=10, n_features=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.root = None
        self.n_features = n_features

    def grow_tree(self, X, y, depth=0):
        n_samples, n_feats = X.shape
        n_classes = len(np.unique(y))

        if (n_classes == 1) or (n_samples < self.min_samples_split) or (depth >= self.max_depth):
            leaf_value = most_value(y)
            return Node(value=leaf_value)

        feature_id = np.random.choice(n_feats, self.n_features, replace=False)
        best_feature, best_threshold = best_split(X, y, feature_id)
        left_node, right_node = split_node(X[best_feature], best_threshold)

        left = self.grow_tree(X.loc[left_node], y.loc[left_node], depth + 1)
        right = self.grow_tree(X.loc[right_node], y.loc[right_node], depth + 1)

        return Node(best_feature, best_threshold, left, right)

    def fit(self, X, y):
        self.n_features = X.shape[1] if self.n_features is None else min(X.shape[1], self.n_features)
        self.root = self.grow_tree(X, y)

    def traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value
        if x[node.feature] <= node.threshold:
            return self.traverse_tree(x, node.left)
        return self.traverse_tree(x, node.right)

    def predict(self, X):
        return np.array([self.traverse_tree(x, self.root) for index, x in X.iterrows()])

In [31]:
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra thủ công
def train_test_split_manual(X, y, test_size=0.2):
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)
    test_set_size = int(X.shape[0] * test_size)

    test_indices = indices[:test_set_size]
    train_indices = indices[test_set_size:]

    return X.iloc[train_indices], X.iloc[test_indices], y.iloc[train_indices], y.iloc[test_indices]

# Chia tập dữ liệu
X_train, X_test, y_train, y_test = train_test_split_manual(X, y, test_size=0.2)

# Tạo và huấn luyện mô hình
tree = DecisionTreeClass(max_depth=5, n_features=3)
tree.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred = tree.predict(X_test)

# Hàm tính độ chính xác
def accuracy(y_actual, y_pred):
    return np.mean(y_actual == y_pred) * 100

# In ra độ chính xác
print(f'Accuracy: {accuracy(y_test, y_pred):.2f}%')

Accuracy: 97.50%
